In [ ]:
import geojson
import rasterio
import icepack, icepack.meshing, icepack.datasets, icepack.plot

### Geometry

First, we'll load in a GeoJSON file describing the outline of Pine Island Ice Shelf.
This outline was hand-digitized from the various input data sets we'll use in a GIS.

In [ ]:
outline_filename = 'pine-island-shelf.geojson'
with open(outline_filename, 'r') as outline_file:
    outline = geojson.load(outline_file)

Next we'll use a few utility functions from icepack to turn this outline into the input format for the mesh generator [gmsh](https://www.gmsh.info).
We can then generate an unstructured triangular mesh of the domain and load in that mesh.

In [ ]:
geometry = icepack.meshing.collection_to_geo(outline)
with open('pine-island-shelf.geo', 'w') as geo_file:
    geo_file.write(geometry.get_code())

In [ ]:
!gmsh -2 -format msh2 -v 2 -o pine-island-shelf.msh pine-island-shelf.geo

In [ ]:
import firedrake
mesh = firedrake.Mesh('pine-island-shelf.msh')

The colors correspond to the numeric IDs of each boundary segment (note the legend in the corner).
We need a way of identifying different boundary conditions in the ice shelf flow model in order to determine where ice is flowing in from and where the terminus is.

In [ ]:
import icepack.plot
fig, axes = icepack.plot.subplots()
icepack.plot.triplot(mesh, axes=axes)
axes.legend();

### Input data

Next we have to load in some observational data for the ice shelf draft.
The demos for icepack include a module to fetch the most common observational data sets.
We'll use BedMachine Antarctica because they've already done the hard work of things like firn and geoid corrections to ice shelf thickness.
This routine will download the BedMachine dataset from NSIDC.
If this is your first time running this notebook, you'll be prompted for your EarthData username and password.

In [ ]:
bedmachine_filename = icepack.datasets.fetch_bedmachine_antarctica()

To get the ice shelf draft, we'll first get the surface elevation and thickness of the ice shelf.

In [ ]:
surface_grid = rasterio.open('netcdf:' + bedmachine_filename + ':surface', 'r')
thickness_grid = rasterio.open('netcdf:' + bedmachine_filename + ':thickness', 'r')

We'll represent the ice shelf draft using continuous, piecewise quadratic basis functions.

In [ ]:
Z = firedrake.FunctionSpace(mesh, family='CG', degree=2)

Next we'll interpolate the gridded data to our finite element space.

In [ ]:
surface = icepack.interpolate(surface_grid, Z)
thickness = icepack.interpolate(thickness_grid, Z)

Finally we can get the ice shelf draft as the difference of the surface and the thickness.

In [ ]:
z_obs = firedrake.interpolate(surface - thickness, Z)

In [ ]:
fig, axes = icepack.plot.subplots()
contours = icepack.plot.tricontourf(z_obs, 40, axes=axes)
fig.colorbar(contours);

In the interest of keeping our numerical solver from losing its bloody mind, we'll smooth over the ice shelf draft a bit.
The weirdest features look to have a radius of about 1km at most.

In [ ]:
from firedrake import inner, grad, dx
z = firedrake.Function(Z)
α = firedrake.Constant(1e3)

J = 0.5 * ((z - z_obs)**2 + α**2 * inner(grad(z), grad(z))) * dx
F = firedrake.derivative(J, z)
firedrake.solve(F == 0, z)

This is enough smoothing to keep the plume solver from exploding, but not so much as to wipe out features like sub-ice shelf channels.

In [ ]:
fig, axes = icepack.plot.subplots()
contours = icepack.plot.tricontourf(z, 40, axes=axes)
fig.colorbar(contours);

Some amount of smoothing is also usually necessary for the ice flow model too.
Newer remote sensing platforms like ICESat-2 are sophisticated enough to resolve individual crevasses, introducing sharp breaks in the ice thickness.
The gradient of the ice thickness is one of the sources of the ice flow model, so these features, which are of too small a scale to really influence the flow by themselves, end up breaking the numerics.

### Initial state

First, we need to decide how we're going to represent the various spatial fields defined on the mesh.
We'll use piecewise constant basis functions in each triangle to keep things as simple as possible.
This basis makes our method is identical to a first-order finite volume method.
To get a higher-order discretization, we could use basis functions with a higher polynomial degree.
We would then need to also use a more accurate timestepping scheme and some kind of flux-limiting scheme.

In [ ]:
Q = firedrake.FunctionSpace(mesh, family='DG', degree=0)
V = firedrake.VectorFunctionSpace(mesh, family='DG', degree=0)

Next, we need to come up with some vaguely sane initial state of the plume.
The model can quickly explode if we initialize it with a weird initial state that has large transients, so we need to find something vaguely reasonable.

In [ ]:
u = firedrake.project(grad(z), V)

In [ ]:
fig, axes = icepack.plot.subplots()
arrows = icepack.plot.quiver(u, axes=axes)
fig.colorbar(arrows);